In [ ]:
import numpy as np
import pandas
import pandas as pd
import talib as ta
import vectorbt as vbt
from numba import njit

# Fetch data
data = pandas.read_csv('BTC-USD1h.csv')
data = data.set_index('Date')
data=data[(data.index > '2020-01-01') & (data.index < '2023-06-30')]
RSI = vbt.IndicatorFactory.from_talib("RSI")
STOCH = vbt.IndicatorFactory.from_talib("STOCH")
# Custom indicator function with Bollinger Bands filter
def custom_indicator(close, high, low, rsi_window=14, fastk_period=9, slowk_period=3,slowd_period=3):
    # Calculate RSI
    rsi = RSI.run(close, rsi_window).real
    # Calculate Stochastic Oscillator
    stoch = STOCH.run(
        high,
        low,
        close,
        fastk_period=fastk_period,
        slowk_period=slowk_period,
        slowk_matype=0,
        slowd_period=slowd_period,
        slowd_matype=0   
    )

    # Convert to numpy arrays
    rsi_values = rsi.to_numpy()
    slowk_values = stoch.slowk.to_numpy()
    slowd_values = stoch.slowd.to_numpy()
    

    # Ensure no NaNs
    rsi_values = np.nan_to_num(rsi_values)
    slowk_values = np.nan_to_num(slowk_values)
    slowd_values = np.nan_to_num(slowd_values)

    # Calculate trend
    trend = np.where(
        (rsi_values>60)&(slowk_values>slowd_values)&(slowk_values>50)&(slowd_values>50) , 1,
        np.where(rsi_values<40, -1, 0)
    )

    return trend

# Define the custom indicator using IndicatorFactory
ind = vbt.IndicatorFactory(
    class_name="Combination",
    short_name="comb",
    input_names=['close', 'high', 'low'],
    param_names=['rsi_window', 'fastk_period', 'slowk_period', 'slowd_period'],
    output_names=["value"]
).from_apply_func(
    custom_indicator,
    rsi_window=32,
    fastk_period=9,
    slowk_period=3,
    slowd_period=3)

# Run the indicator
res = ind.run(
    data['close'],
    data['high'],
    data['low'],
    rsi_window=32,
    fastk_period=9,
    slowk_period=3,
    slowd_period=3
)

# Generate entries and exits signals
entries = res.value == 1
exits = res.value == -1

# Create a portfolio from the signals
pf = vbt.Portfolio.from_signals(data['close'], entries, exits,freq='1h',direction='both',fees=0.04/100,sl_stop=0.05)
# 显示回测结果
print(pf.stats())
pf.plot().show()


In [40]:
import numpy as np
import pandas
data = pandas.read_csv('成發輔助資料/BTCUSDT_1d.csv')
data = data.set_index('open_time')
data.index.names = ['Date']
# Delete a single column
data = data.drop(['close_time','Unnamed: 0'], axis=1)
print(data)

                open      high       low     close      volume  quote_volume  \
Date                                                                           
2020-01-01   7189.43   7260.43   7170.15   7197.57   56801.329  4.096788e+08   
2020-01-02   7197.57   7209.59   6922.00   6962.04  115295.677  8.156278e+08   
2020-01-03   6962.34   7407.28   6863.44   7341.72  208493.458  1.507314e+09   
2020-01-04   7341.60   7400.00   7269.21   7350.71   92586.033  6.784024e+08   
2020-01-05   7350.54   7495.00   7303.00   7354.36  117765.972  8.744885e+08   
...              ...       ...       ...       ...         ...           ...   
2024-06-26  61777.90  62465.10  60696.00  60860.10  170601.180  1.047422e+10   
2024-06-27  60860.10  62379.70  60605.00  61682.00  160172.482  9.845346e+09   
2024-06-28  61682.10  62204.00  60038.00  60429.80  195508.125  1.196316e+10   
2024-06-29  60429.80  61232.00  60400.10  60982.40   79680.289  4.854467e+09   
2024-06-30  60982.50  63058.40  60705.00

In [41]:
data.to_csv('BTC-USD1d.csv')

In [ ]:
data = vbt.YFData.download(
    "BTC-USD",
    missing_index='drop',
    start='2020-01-01',
    end='2022-08-30',
    interval="1D"
)

In [11]:
 # Calculate Bollinger Bands
bbands = vbt.BBANDS.run(
    data.get('Close'),
    window=20,
    ewm=False,
    alpha=2,
    short_name='bb',
    hide_params=None,
    hide_default=True,
)
lower_band = bbands.lower
middle_band = bbands.middle
print(lower_band)
print(middle_band)
print(data.get('Close'))

Date
2019-12-31 00:00:00+00:00             NaN
2020-01-01 00:00:00+00:00             NaN
2020-01-02 00:00:00+00:00             NaN
2020-01-03 00:00:00+00:00             NaN
2020-01-04 00:00:00+00:00             NaN
                                 ...     
2022-08-25 00:00:00+00:00    20477.622860
2022-08-26 00:00:00+00:00    20082.662578
2022-08-27 00:00:00+00:00    19683.392891
2022-08-28 00:00:00+00:00    19253.070729
2022-08-29 00:00:00+00:00    19001.329012
Freq: D, Name: (20, False, 2, Close), Length: 973, dtype: float64
Date
2019-12-31 00:00:00+00:00             NaN
2020-01-01 00:00:00+00:00             NaN
2020-01-02 00:00:00+00:00             NaN
2020-01-03 00:00:00+00:00             NaN
2020-01-04 00:00:00+00:00             NaN
                                 ...     
2022-08-25 00:00:00+00:00    22911.601074
2022-08-26 00:00:00+00:00    22776.538086
2022-08-27 00:00:00+00:00    22619.830469
2022-08-28 00:00:00+00:00    22410.196875
2022-08-29 00:00:00+00:00    22266.880664
